In [1]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import datetime as dt
from random import seed
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K
tf.compat.v1.experimental.output_all_intermediates(True)
tf.config.run_functions_eagerly(True)


In [2]:
seed(36)

In [3]:
DATASET_FILE = 'compiled_data_jer_edit 2017.csv'



In [24]:
df = pd.read_csv(DATASET_FILE)
df.head(10)

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x
0,0,0,0,1,1,0,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 00:00:00,0.0,14420
1,1,365,0,0,0,1,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 01:00:00,3600.0,14420
2,2,730,0,0,0,0,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 02:00:00,7200.0,14420
3,3,1095,0,0,1,1,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 03:00:00,10800.0,14420
4,4,1460,0,0,0,0,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 04:00:00,14400.0,14420
5,5,1825,0,0,1,0,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 05:00:00,18000.0,14420
6,6,2190,0,0,0,0,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 06:00:00,21600.0,14420
7,7,2555,0,0,0,0,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 07:00:00,25200.0,14420
8,8,2920,0,0,1,0,0,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 08:00:00,28800.0,14420
9,9,3285,6,5,20,4,20,12.53,21.39,22.695466,25.325458,826.98,641.7,0.027444,0.039466,2017-01-01 09:00:00,32400.0,14420


In [5]:
df = df[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'Velocity_Montalban', 'Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']]
df.head(10)

,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Velocity_Montalban,Waterlevel_Sto_Nino,Velocity_Sto_Nino
0,0,1,1,0,0,0.039466,12.53,0.027444
1,0,0,0,1,0,0.039466,12.53,0.027444
2,0,0,0,0,0,0.039466,12.53,0.027444
3,0,0,1,1,0,0.039466,12.53,0.027444
4,0,0,0,0,0,0.039466,12.53,0.027444
5,0,0,1,0,0,0.039466,12.53,0.027444
6,0,0,0,0,0,0.039466,12.53,0.027444
7,0,0,0,0,0,0.039466,12.53,0.027444
8,0,0,1,0,0,0.039466,12.53,0.027444
9,6,5,20,4,20,0.039466,12.53,0.027444


In [6]:
# Splitting for time series: split into 70-20-10
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

In [7]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
    
    # properties to access them as tf datasets
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [8]:
# The wide window uses independent hours of data as input to predict the water level of the next hour
# Here, the prediction is done on 6 hours
# This is used for Dense and Recurrent Neural Networks
wide_window = WindowGenerator(
        input_width=6, label_width=6, shift=1,
        label_columns=['Velocity_Sto_Nino', 'Waterlevel_Sto_Nino']
    )

wide_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [1 2 3 4 5 6]
Label column name(s): ['Velocity_Sto_Nino', 'Waterlevel_Sto_Nino']

In [9]:
wide_window.train.take(1)

c:\users\jeremy tan\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


<_TakeDataset element_spec=(TensorSpec(shape=(None, 6, 8), dtype=tf.float32, name=None), TensorSpec(shape=(None, 6, 2), dtype=tf.float32, name=None))>

In [10]:
# The conv window is used for the Convolutional Neural Netwrok
# 6 consecutive hours of data are used together to make predictions one hour into the future
CONV_WIDTH = 6
conv_window = WindowGenerator(
        input_width=CONV_WIDTH,
        label_width=1,
        shift=1,
        label_columns=['Velocity_Sto_Nino', 'Waterlevel_Sto_Nino']
    )

conv_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [6]
Label column name(s): ['Velocity_Sto_Nino', 'Waterlevel_Sto_Nino']

In [11]:
conv_window.train.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(None, 6, 8), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 2), dtype=tf.float32, name=None))>

In [12]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [13]:
def NSE(y_true, y_pred):
    '''
    This is the Nash-Sutcliffe Efficiency Coefficient
    '''
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [14]:
#PDE Loss Function
def custom_loss(grads_inputs, fric_coeff=0.033, slope=1/1500):
    du_dx, du_dt, dh_dx= grads_inputs[:,0], grads_inputs[:,1], grads_inputs[:,2]
    g = K.constant(9.8)
    
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
        loss_saint_venant = du_dt + y_pred[:,0] * du_dx + g*dh_dx + g*slope + g*K.square(fric_coeff) * K.square(y_true[:,0])/(K.pow(y_true[:,1], 4/3) + K.epsilon())
        l = K.mean(K.square(loss_saint_venant))

        return 2*l+ K.sum(K.mean(K.square(y_pred - y_true), axis=0))
   
    # Return a function
    return loss

In [15]:
# Automatic Differentiation
def get_derivative(model):
    grads_u = K.gradients(model.output[:,0], [14420, 1440])[0]
    grads_h = K.gradients(model.output[:,1], [14420, 1440])[0]
    
    du_dx, du_dt, dh_dx = grads_u[:,0],grads_u[:,1],grads_h[:,0]
    
    return K.stack((du_dx, du_dt, dh_dx), axis=1)

In [16]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Example: Scalar function for which we want gradients
x = tf.Variable(3.0)
y = tf.Variable(4.0)

# Define a simple loss function: z = x^2 + y^2
z = x**2 + y**2

# Compute the gradients of `z` with respect to `x` and `y`
gradients = K.gradients(z, [x, y])



RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [ ]:
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    gradient_values = sess.run(gradients)
    print("Gradient values:", gradient_values)

In [ ]:
# For easy compiling and fitting of different models
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    calc_grad_inputs = get_derivative(model)
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    model.compile(
        loss=[custom_loss(calc_grad_inputs)], 
        optimizer=tf.keras.optimizers.Adam(), 
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )

    history = model.fit(
        window.train, 
        epochs=MAX_EPOCHS,
        validation_data=window.val,
        callbacks=[early_stopping]
    )

    return history

In [17]:
# Dense Neural Network
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=(6,10)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
])

# Convolution Neural Network
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(CONV_WIDTH,), activation='relu', input_shape=(6,10)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2),
])

# LSTM
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(6,10)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
])

## DeepXDE Package

https://www.youtube.com/watch?v=EO2lc4tXBHA

Saint Venant Momentum Equation

Two Functions:

1. $u_i(x,t)$ is the flow velocity
2. $h_i(x,t)$ is the water height.

$$
\frac{\partial u_i}{\partial t} + u_i \frac{\partial u_i}{\partial x} + g \frac{\partial h_i}{\partial x} - g S_0 + g S_f = 0
$$

In [18]:
import win32ui
import deepxde as dde

Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


No backend selected.
Finding available backend...
Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
Instructions for updating:
non-resource variables are not supported in the long term


In [19]:
geom = dde.geometry.Interval(0, 14420) # interval for x
timedomain = dde.geometry.TimeDomain(0, 31532400) # interval for t
geomtime = dde.geometry.GeometryXTime(geom, timedomain)


In [ ]:
# x is 2-D tensor array
# y is the output

def pde(x,y):
    dy_t = dde.grad.jacobian(y,x, j = 1)
    dy_xx = dde.grad.hessian(y,x,j = 0)
    
    return (dy_t - dy_xx*0.3)

In [32]:
# Create the 2-D Tensor

# Define the range and shape of x and t
t_values = tf.range(start=0, limit=31532400 + 3600, delta=3600) 
x_values = tf.fill(tf.shape(t_values), 14420)

# Create a 2-D tensor array using meshgrid
X, T = tf.meshgrid(x_values, t_values)

# Combine X and T into a single tensor
tensor_2d = tf.stack([X, T], axis=-1)  
 

# Print the resulting tensor
print("2-D Tensor Array (shape {}):\n".format(tensor_2d.shape), tensor_2d)

2-D Tensor Array (shape (8760, 8760, 2)):
 Tensor("stack_4:0", shape=(8760, 8760, 2), dtype=int32)


In [33]:
# Condition Equations
def initial_condition(x):
    return np.array([12.53, 0.027444]) # initial depth of 12.53 m and water velocity of 0.027444 m/s

<tf.Tensor 'stack_4:0' shape=(8760, 8760, 2) dtype=int32>

In [22]:
dense.output[:,1]

<tf.Tensor 'strided_slice:0' shape=(None, 2) dtype=float32>

In [23]:
dense.output[:,0]

<tf.Tensor 'strided_slice_1:0' shape=(None, 2) dtype=float32>

In [24]:
dense.input[:,0]

<tf.Tensor 'strided_slice_2:0' shape=(None, 10) dtype=float32>

In [25]:
conv_model.input[0][0]

<tf.Tensor 'strided_slice_4:0' shape=(10,) dtype=float32>

In [26]:
conv_model.output

<tf.Tensor 'dense_6/BiasAdd:0' shape=(None, 1, 2) dtype=float32>

In [27]:
dense_history = compile_and_fit(dense, wide_window)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
conv_history = compile_and_fit(conv_model, conv_window)

In [ ]:
lstm_history = compile_and_fit(lstm_model, wide_window)

In [ ]:
val_performance = {}
performance = {}

In [ ]:
val_performance['Dense'] = dense.evaluate(wide_window.val)

In [ ]:
performance['Dense'] = dense.evaluate(wide_window.test, verbose=0)

In [ ]:
val_performance['Conv'] = conv_model.evaluate(conv_window.val)

In [ ]:
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [ ]:
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)

In [ ]:
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
val_performance

In [ ]:
performance